In [11]:
#preprocess the images 
import os
import cv2
import numpy as np

def preprocess_underwater_image(image):
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    l_channel, a_channel, b_channel = cv2.split(lab_image)

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced_l_channel = clahe.apply(l_channel)

    enhanced_lab_image = cv2.merge([enhanced_l_channel, a_channel, b_channel])

    enhanced_image = cv2.cvtColor(enhanced_lab_image, cv2.COLOR_LAB2BGR)

    gamma = 1.5
    enhanced_image = np.power(enhanced_image / 255.0, gamma)
    enhanced_image = np.uint8(enhanced_image * 255)

    return enhanced_image

def preprocess_images_in_folder(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(input_folder):
        input_path = os.path.join(input_folder, filename)
        if os.path.isfile(input_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image = cv2.imread(input_path)
            preprocessed_image = preprocess_underwater_image(image)
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, preprocessed_image)
            print(f"Processed and saved: {output_path}")

# Define input and output folders
input_folder = r'DePondFi24_Train\DePoondFi24_Train\Images'
output_folder = r'preprocessed_images'

preprocess_images_in_folder(input_folder, output_folder)


Processed and saved: preprocessed_images\frame120_jpg.rf.08bec5e869c708b0da1cbe514c645ca9.jpg
Processed and saved: preprocessed_images\frame120_jpg.rf.3fade6ea16a9f21b1c75d46195dcbcf9.jpg
Processed and saved: preprocessed_images\frame120_jpg.rf.4783b65913220f1faf185952cc71f7f2.jpg
Processed and saved: preprocessed_images\frame120_jpg.rf.66f91f5e163c7f721bddd57660866af7.jpg
Processed and saved: preprocessed_images\frame120_jpg.rf.972c05f94e5ad3a60ba28127970fa4c0.jpg
Processed and saved: preprocessed_images\frame120_jpg.rf.9a1736782243ca59cb5f401670b3ce80.jpg
Processed and saved: preprocessed_images\frame122_jpg.rf.2af2993938c997bc32377028f1e1f9e7.jpg
Processed and saved: preprocessed_images\frame122_jpg.rf.7d74cc9befdf8f75b7ad92faded636c2.jpg
Processed and saved: preprocessed_images\frame122_jpg.rf.e7eb1250d321e81932f5cfaa3525c2f8.jpg
Processed and saved: preprocessed_images\frame124_jpg.rf.10bac02292dc819c365171299e9e4cf2.jpg
Processed and saved: preprocessed_images\frame129_jpg.rf.4de

In [12]:
#converting anotations to yolo format
# Set the paths for the image and keypoint folders
keypoint_folder = 'DePondFi24_Train\DePoondFi24_Train\Keypoints'

# Set the output folder for the YOLO annotations
output_folder = 'YOLO_annotations_updated'
height = 640
width = 640
os.makedirs(output_folder, exist_ok=True)
print("Created the output folder")
# Loop through all images in the image folder
for filename in os.listdir(input_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path)

        base_name = os.path.splitext(filename)[0]
        keypoint_file = os.path.join(keypoint_folder, f'{base_name}.txt')

        keypoints = []
        with open(keypoint_file, 'r') as f:
            for line in f:
                x, y = map(float, line.strip().split(','))
                keypoints.append((x, y))

        output_file = os.path.join(output_folder, f'{base_name}.txt')
        with open(output_file, 'w') as f:
            for i in range(0, len(keypoints), 9):
                # Extract the x and y coordinates
                x_coords = [x for x, y in keypoints[i:i+9]]
                y_coords = [y for x, y in keypoints[i:i+9]]
                # Calculate the bounding box coordinates
                fish_center_x, fish_center_y = x_coords[3] , y_coords[3]
                # Calculate the center of the bounding box
                x_center = fish_center_x / width
                y_center = fish_center_y / height
                bbw = (max(x_coords) - min(x_coords)) / width
                bbh = (max(y_coords) - min(y_coords)) / height
                # Calculate the width and height of the bounding box
                # Write the annotation to the file
                mouth_x , mouth_y = x_coords[0]/width, y_coords[0]/height
                eye_x, eye_y = x_coords[1]/width, y_coords[1]/height
                top_fin_x, top_fin_y = x_coords[2]/width, y_coords[2]/height
                fish_center_x , fish_center_y = fish_center_x/width, fish_center_y/height
                bottom_center_x , bottom_center_y = x_coords[4]/width , y_coords[4]/height
                start_point_tail_x, start_point_tail_y = x_coords[5]/width, y_coords[5]/height
                top_outline_tail_x , top_outline_tail_y = x_coords[6]/width, y_coords[6]/height
                mid_outline_tail_x , mid_outline_tail_y = x_coords[7]/width, y_coords[7]/height
                bottom_outline_tail_x , bottom_outline_tail_y = x_coords[8]/width, y_coords[8]/height
                f.write(f'0 {x_center} {y_center} {bbw} {bbh} {mouth_x} {mouth_y} {eye_x} {eye_y} {top_fin_x} {top_fin_y} {fish_center_x} {fish_center_y} {bottom_center_x} {bottom_center_y} {start_point_tail_x} {start_point_tail_y} {top_outline_tail_x} {top_outline_tail_y} {mid_outline_tail_x} {mid_outline_tail_y} {bottom_outline_tail_x} {bottom_outline_tail_y}\n')


Created the output folder


In [13]:
#dividing the images into train and test for YOlO 
import os
import random
import shutil
from sklearn.model_selection import train_test_split

# Paths
images_dir = 'preprocessed_images'
annotations_dir = 'YOLO_annotations_updated'
output_dir = 'YOLO_training/'

# Create output directories
train_images_dir = os.path.join(output_dir, 'images', 'train')
train_labels_dir = os.path.join(output_dir, 'labels', 'train')
val_images_dir = os.path.join(output_dir, 'images', 'val')
val_labels_dir = os.path.join(output_dir, 'labels', 'val')

os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)

image_files = [f for f in os.listdir(images_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Split the dataset into train and test
train_files, test_files = train_test_split(image_files, test_size=0.2, random_state=42)

def copy_files(files, src_dir, dst_dir, extension):
    for file in files:
        name, _ = os.path.splitext(file)
        src_path = os.path.join(src_dir, f"{name}{extension}")
        dst_path = os.path.join(dst_dir, f"{name}{extension}")
        if os.path.exists(src_path):
            shutil.copy(src_path, dst_path)

copy_files(train_files, images_dir, train_images_dir, '.jpg')
copy_files(train_files, annotations_dir, train_labels_dir, '.txt')

copy_files(test_files, images_dir,val_images_dir, '.jpg')
copy_files(test_files, annotations_dir, val_labels_dir, '.txt')

print("Dataset split and copied successfully.")


Dataset split and copied successfully.


In [14]:
#create a data.yaml file
data_yaml = f"""
# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: YOLO_training # dataset root dir
train: images/train  # train images (relative to 'path') 4 images
val: images/val  # val images (relative to 'path') 4 images

# Keypoints
kpt_shape: [9, 2]  # number of keypoints, number of dims (2 for x,y or 3 for x,y,visible)

# Classes dictionary
names:
  0: fish
"""

# Save the data.yaml file
directory = 'YOLO_training'
file_name = 'data.yaml'
file_path = os.path.join(directory, file_name)

with open(file_path, 'w') as f:
    f.write(data_yaml)
print("data.yaml file created successfully.")

#Also ensure that in the settings.yaml file is created and datasets_dir is set to the directory containing the YOLO_training folder.
#The location of this file varies depending on the installation method, but it is typically located inside AppData/Roaming/Roboflow/settings.yaml on Windows or ~/.roboflow/settings.yaml on Linux or MacOS.
#The settings.yaml file should look like this:
'''
settings_version: 0.0.4
datasets_dir: path_to_source_directory\Pondify\
weights_dir: weights
runs_dir: runs
uuid: xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx
sync: true
api_key: ''
openai_api_key: ''
clearml: true
comet: true
dvc: true
hub: true
mlflow: true
neptune: true
raytune: true
tensorboard: true
wandb: true
'''


data.yaml file created successfully.


"\nsettings_version: 0.0.4\ndatasets_dir: path_to_source_directory\\Pondifyweights_dir: weights\nruns_dir: runs\nuuid: xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx\nsync: true\napi_key: ''\nopenai_api_key: ''\nclearml: true\ncomet: true\ndvc: true\nhub: true\nmlflow: true\nneptune: true\nraytune: true\ntensorboard: true\nwandb: true\n"

In [15]:
end_time = time.time()
total_time = end_time - start_time
print(f"Total time taken: {end_time - start_time} seconds")


Total time taken: 30.470528841018677 seconds


In [2]:
import torch
from ultralytics import YOLO

# Check if CUDA is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

model = YOLO("yolov8n-pose.yaml").load("yolov8n-pose.pt")  # build from YAML and transfer weights
#change the paths accordingly
results = model.train(data="YOLO_training\data.yaml", epochs=100, imgsz=640,device=device,batch = 40, plots = True, box = 12.0, pose = 15.0,patience = 10, cos_lr = True, project = "Pondify", name = "yolov8n-pose_test_11", exist_ok = True)

Using device: cuda
Transferred 397/397 items from pretrained weights
New https://pypi.org/project/ultralytics/8.2.32 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.12  Python-3.11.5 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
engine\trainer: task=pose, mode=train, model=yolov8n-pose.yaml, data=YOLO_training\data.yaml, epochs=100, time=None, patience=10, batch=40, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=Pondify, name=yolov8n-pose_test_11, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualiz

train: Scanning C:\Users\sudee\Documents\Projects\Pondify\YOLO_training\labels\train.cache... 448 images, 0 backgrounds, 0 corrupt: 100%|██████████| 448/448 [00:00<?, ?it/s]

WARNING  No 'flip_idx' array defined in data.yaml, setting augmentation 'fliplr=0.0'



val: Scanning C:\Users\sudee\Documents\Projects\Pondify\YOLO_training\labels\val.cache... 112 images, 0 backgrounds, 0 corrupt: 100%|██████████| 112/112 [00:00<?, ?it/s]


Plotting labels to Pondify\yolov8n-pose_test_11\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 63 weight(decay=0.0), 73 weight(decay=0.000625), 72 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to Pondify\yolov8n-pose_test_11
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


...

     98/100      5.66G      1.281      2.168          0     0.4884      1.048         28        640: 100%|██████████| 12/12 [00:03<00:00,  3.18it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.64it/s]

                   all        112        309      0.894      0.795       0.88      0.528       0.85      0.757      0.802      0.457
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 88, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



98 epochs completed in 0.162 hours.
Optimizer stripped from Pondify\yolov8n-pose_test_11\weights\last.pt, 6.4MB
Optimizer stripped from Pondify\yolov8n-pose_test_11\weights\best.pt, 6.4MB

Validating Pondify\yolov8n-pose_test_11\weights\best.pt...
Ultralytics YOLOv8.2.12  Python-3.11.5 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
YOLOv8n-pose summary (fused): 187 layers, 3088301 parameters, 0 gradients, 8.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.30it/s]


                   all        112        309      0.829      0.825      0.886      0.535      0.858      0.726      0.811      0.463
Speed: 0.2ms preprocess, 2.1ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to Pondify\yolov8n-pose_test_11


In [24]:
import os 
import cv2

#traversing through the images and predicting the bounding boxes

# Load the model
#change the paths accordingly
model = YOLO(r"Pondify\yolov8n-pose_test_11\weights\last.pt")  # load a custom model

images_dir = r"preprocessed_images"
image_files = [f for f in os.listdir(images_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

os.makedirs("results", exist_ok=True)

os.makedirs("keypoint_predictions", exist_ok=True)

for image_file in image_files:
    image_path = os.path.join(images_dir, image_file)

    results = model(image_path)

    output_path = os.path.join("results", image_file)

    for result in results:
        result.save(filename=output_path)   

    print(f"Processed and saved: {output_path}")


    output_path = os.path.join("keypoint_predictions", f"{os.path.splitext(image_file)[0]}.txt")
    with open(output_path, "w") as file:
        for result in results:
            keypoints = result.keypoints  
            for i in range(keypoints.xy.shape[0]):
                for j in range(keypoints.xy.shape[1]):
                    x = keypoints.xy[i][j][0]
                    y = keypoints.xy[i][j][1]
                    file.write(f"{float(x)},{float(y)}\n")
        print(f"Processed and saved: {output_path}")
            


image 1/1 c:\Users\sudee\Documents\Projects\Pondify\preprocessed_images\frame120_jpg.rf.08bec5e869c708b0da1cbe514c645ca9.jpg: 640x640 4 fishs, 44.1ms
Speed: 4.0ms preprocess, 44.1ms inference, 8.0ms postprocess per image at shape (1, 3, 640, 640)
Processed and saved: results\frame120_jpg.rf.08bec5e869c708b0da1cbe514c645ca9.jpg
Processed and saved: keypoint_predictions\frame120_jpg.rf.08bec5e869c708b0da1cbe514c645ca9.txt

image 1/1 c:\Users\sudee\Documents\Projects\Pondify\preprocessed_images\frame120_jpg.rf.3fade6ea16a9f21b1c75d46195dcbcf9.jpg: 640x640 3 fishs, 28.0ms
Speed: 3.0ms preprocess, 28.0ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)
Processed and saved: results\frame120_jpg.rf.3fade6ea16a9f21b1c75d46195dcbcf9.jpg
Processed and saved: keypoint_predictions\frame120_jpg.rf.3fade6ea16a9f21b1c75d46195dcbcf9.txt

image 1/1 c:\Users\sudee\Documents\Projects\Pondify\preprocessed_images\frame120_jpg.rf.4783b65913220f1faf185952cc71f7f2.jpg: 640x640 3 fishs, 30.0ms